In [2]:
# %pip install tensorflow-addons
# %pip uninstall tensorflow-addons
%pip install tensorflow==2.15.0 tensorflow-addons

  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.8/208.8 MB 5.5 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 4.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.6.0
    Uninstalling keras-3.6.0:
      Successfully uninstalled keras-3.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
    

In [124]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sn

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, GroupShuffleSplit 

import glob
import sys
import os
import math
import gc
import sys
import sklearn
import scipy

# print(f'Tensorflow V{tf.__version__}')
# print(f'Keras V{tf.keras.__version__}')
# print(f'Python V{sys.version}')

# LSTM

In [21]:
words = [
    'หนังสือ', 'ทารก', 'อยู่', 'อาหาร', 'พูด', 'คน', 'ยินดีด้วย', 'ผู้หญิง',
    'วันอังคาร', 'วันเสาร์', 'ต้นไม้', 'ทำไม', 'อ่าน', 'ฟัง', 'ภาษามือไทย', 'กระโดด',
    'จาน', 'วันศุกร์', 'แมว', 'ดื่ม', 'คุณ', 'ผลไม้', 'กลางคืน', 'ช้อน',
    'ร้องไห้', 'ตื่น', 'ดอกไม้', 'ของขวัญ', 'คอมพิวเตอร์', 'สัตว์', 'วันพฤหัสบดี', 'วันจันทร์',
    'ยืน', 'เล่น', 'ยิ้ม', 'ไม่ใช่', 'ทะเล', 'เขา', 'หมา', 'กิน', 'โรงเรียน',
    'ฉัน', 'เพื่อน', 'ที่ไหน', 'เด็ก', 'ภาษาไทย', 'บ้าน', 'นม', 'ปิด', 'วันอาทิตย์',
    'ไม่เป็นไร', 'ขอบคุณ', 'เปิด', 'ลาก่อน', 'ทำ', 'ผู้ชาย', 'ภูเขา', 'ใคร', 'ท้องฟ้า',
    'เธอ', 'ครอบครัว', 'เขียน', 'สบายดี', 'เดิน', 'นั่ง', 'วันพุธ', 'แตงโม', 'ล้าง', 'อย่างไร'
]


In [125]:
words = np.array(os.listdir('/Users/theerat/Downloads/MP_DATA3'))

In [126]:
DATA_PATH = os.path.join('/Users/theerat/Downloads/MP_DATA3')

sequence_length = 30
no_sequences = 30

In [127]:
label_map = {label:num for num, label in enumerate(words)}

In [128]:
label_map

{'พลเอกอนุพงษ์ เผ่าจินดา': 0,
 'เนื้อหมู': 1,
 'เจอ': 2,
 'สวัสดี': 3,
 'ยก': 4,
 'อยู่': 5,
 'อะไร': 6,
 'ณัฐวุฒิ ใสยเกื้อ': 7,
 'สุชัชวีร์ สุวรรณสวัสดิ์': 8,
 'ไพบูลย์ นิติตะวัน': 9,
 'อนุทิน ชาญวีรกูล': 10,
 'เฝ้า': 11,
 'ทักษิณ ชินวัตร': 12,
 'ชวน หลีกภัย': 13,
 'rain': 14,
 '.DS_Store': 15,
 'ตัด': 16,
 'พิธา ลิ้มเจริญรัตน์': 17,
 'ขึ้น': 18,
 'แพทองธาร ชินวัตร': 19,
 'ช่วย': 20,
 'ศักดิ์สยาม ชิดชอบ': 21,
 'กรณ์ จาติกวณิช': 22,
 'คุณสบายดีไหม': 23,
 'ฟัง': 24,
 'today': 25,
 'ยิ่งลักษณ์ ชินวัตร': 26,
 'กระโดด': 27,
 'พล.ต.อ.เสรีพิศุทธ์ เตมียเวสวราวุธ ศิลปะอาชา': 28,
 'คุย': 29,
 'ชาดา ไทยเศรษฐ์': 30,
 'วาง': 31,
 'พลเอกประยุทธ์ จันทร์โอชา': 32,
 'จำได้': 33,
 'ดูเหมือน': 34,
 'พลเอกประวิตร วงษ์สุวรรณ': 35,
 'เศรษฐา ทวีสิน': 36,
 'จุรินทร์ ลักษณวิศิษฏ์': 37,
 'ฟ': 38,
 'ธ': 39,
 'ม': 40,
 'ศ': 41,
 'ฑ': 42,
 'ถ': 43,
 'ค': 44,
 'ชน': 45,
 'ช': 46,
 'ญ': 47,
 'ธรรมนัส พรหมเผ่า': 48,
 'ฮ': 49,
 'ท': 50,
 'ษ': 51,
 'ฐ': 52,
 'น': 53,
 'ภ': 54,
 'พ': 55,
 'ลง': 56,
 'ว': 57,
 'ซ': 58,


อัดไฟล์

In [157]:
filtered_words = np.array(list(filter(lambda word: word != '.DS_Store', words)))

In [158]:
filtered_words

array(['พลเอกอนุพงษ์ เผ่าจินดา', 'เนื้อหมู', 'เจอ', 'สวัสดี', 'ยก',
       'อยู่', 'อะไร', 'ณัฐวุฒิ ใสยเกื้อ', 'สุชัชวีร์ สุวรรณสวัสดิ์',
       'ไพบูลย์ นิติตะวัน', 'อนุทิน ชาญวีรกูล', 'เฝ้า', 'ทักษิณ ชินวัตร',
       'ชวน หลีกภัย', 'rain', 'ตัด', 'พิธา ลิ้มเจริญรัตน์', 'ขึ้น',
       'แพทองธาร ชินวัตร', 'ช่วย', 'ศักดิ์สยาม ชิดชอบ', 'กรณ์ จาติกวณิช',
       'คุณสบายดีไหม', 'ฟัง', 'today', 'ยิ่งลักษณ์ ชินวัตร', 'กระโดด',
       'พล.ต.อ.เสรีพิศุทธ์ เตมียเวสวราวุธ ศิลปะอาชา', 'คุย',
       'ชาดา ไทยเศรษฐ์', 'วาง', 'พลเอกประยุทธ์ จันทร์โอชา', 'จำได้',
       'ดูเหมือน', 'พลเอกประวิตร วงษ์สุวรรณ', 'เศรษฐา ทวีสิน',
       'จุรินทร์ ลักษณวิศิษฏ์', 'ฟ', 'ธ', 'ม', 'ศ', 'ฑ', 'ถ', 'ค', 'ชน',
       'ช', 'ญ', 'ธรรมนัส พรหมเผ่า', 'ฮ', 'ท', 'ษ', 'ฐ', 'น', 'ภ', 'พ',
       'ลง', 'ว', 'ซ', 'เชื่อ', 'ข', 'รวม', 'มอง', 'รังสิมันต์ โรม',
       'ทำหาย', 'ไตรรงค์ สุวรรณคีรี', 'cold', 'กลับ', 'ขนม', 'ปลูก',
       'พนักงานขาย', 'ผ่าน', 'สมัคร', 'ตก', 'ขอบคุณ', 'เจ้าหน้าที่ตำรวจ',
       'พา', 'พีระพันธุ์ 

In [131]:
# ใช้สำหรับไฟล์ของเรา
sequences, labels = [], []
for action in words:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            csv_path = os.path.join(DATA_PATH, action, str(sequence), f'{frame_num}.csv')
            print(f'Loading: {csv_path}')
            res = pd.read_csv(csv_path)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

Loading: /Users/theerat/Downloads/MP_DATA3/พลเอกอนุพงษ์ เผ่าจินดา/0/0.csv


FileNotFoundError: [Errno 2] No such file or directory: '/Users/theerat/Downloads/MP_DATA3/พลเอกอนุพงษ์ เผ่าจินดา/0/0.csv'

In [132]:
sequences, labels = [], []
for action in filtered_words:
    print(action)
    length = os.listdir(f'/Users/theerat/Downloads/MP_DATA3/{action}')
    # print(length)
    for sequence in length:
        if sequence == '.DS_Store':
            continue
        window = []
        sequence_length = os.listdir(f'/Users/theerat/Downloads/MP_DATA3/{action}/{sequence}')
        sequence_length = [filename.replace('.npy', '') for filename in sequence_length if filename.endswith('.npy')]
        for frame_num in sequence_length:
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

พลเอกอนุพงษ์ เผ่าจินดา
เนื้อหมู
เจอ
สวัสดี
ยก
อยู่
อะไร
ณัฐวุฒิ ใสยเกื้อ
สุชัชวีร์ สุวรรณสวัสดิ์
ไพบูลย์ นิติตะวัน
อนุทิน ชาญวีรกูล
เฝ้า
ทักษิณ ชินวัตร
ชวน หลีกภัย
rain
ตัด
พิธา ลิ้มเจริญรัตน์
ขึ้น
แพทองธาร ชินวัตร
ช่วย
ศักดิ์สยาม ชิดชอบ
กรณ์ จาติกวณิช
คุณสบายดีไหม
ฟัง
today
ยิ่งลักษณ์ ชินวัตร
กระโดด
พล.ต.อ.เสรีพิศุทธ์ เตมียเวสวราวุธ ศิลปะอาชา
คุย
ชาดา ไทยเศรษฐ์
วาง
พลเอกประยุทธ์ จันทร์โอชา
จำได้
ดูเหมือน
พลเอกประวิตร วงษ์สุวรรณ
เศรษฐา ทวีสิน
จุรินทร์ ลักษณวิศิษฏ์
ฟ
ธ
ม
ศ
ฑ
ถ
ค
ชน
ช
ญ
ธรรมนัส พรหมเผ่า
ฮ
ท
ษ
ฐ
น
ภ
พ
ลง
ว
ซ
เชื่อ
ข
รวม
มอง
รังสิมันต์ โรม
ทำหาย
ไตรรงค์ สุวรรณคีรี
cold
กลับ
ขนม
ปลูก
พนักงานขาย
ผ่าน
สมัคร
ตก
ขอบคุณ
เจ้าหน้าที่ตำรวจ
พา
พีระพันธุ์ สาลีรัฐวิภาค
ศิริกัญญา ตันสกุล
พ่อครัว
หาย
ชัยวุฒิ ธนาคมานุสรณ์
คุณหญิงสุดารัตน์ เกยุราพันธุ์
นักเขียนโปรแกรม
นพ.ชลน่าน ศรีแก้ว
ง
แนะนำ
ฎ
พริษฐ์ วัชรสินธุ
ฉ
ป
ย
ผ
นฤมล ภิญโญสินวัฒน์
ล
ฬ
ต
ห
ฒ
จ
ฏ
แบก
ฆ
ก
เนื้อลูกแกะ
ส
ณ
อ
สุวัจน์ ลิปตพัลลภ
รอ
ด
ฝ
บ
ร


In [159]:
df = pd.DataFrame({'sequence':sequences, 'label':labels})
df

,sequence,label
0,"[[0.456181138753891, 0.4181082844734192, -0.49...",0
1,"[[0.4541245996952057, 0.41514167189598083, -0....",0
2,"[[0.46002864837646484, 0.38592401146888733, -0...",0
3,"[[0.45991572737693787, 0.41418561339378357, -0...",0
4,"[[0.44606512784957886, 0.4121881425380707, -0....",0
...,...,...
1250,"[[0.5187767744064331, 0.43978872895240784, -0....",113
1251,"[[0.521430492401123, 0.4260687232017517, -0.52...",113
1252,"[[0.5231441855430603, 0.4265885651111603, -0.5...",113
1253,"[[0.5232256650924683, 0.4244484007358551, -0.5...",113


In [160]:
for seq in sequences:
    print([s.shape for s in seq])

[(1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,)]
[(1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,)]
[(1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,)]
[(1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,), (1662,

In [161]:
max_length = max(len(seq) for seq in sequences)

In [162]:
padded_sequences = np.array([np.pad(seq, ((0, max_length - len(seq)), (0, 0)), mode='constant') if len(seq) < max_length else seq for seq in sequences])

In [163]:
for rec in labels:
    print([r.shape for r in rec])

TypeError: 'int' object is not iterable

In [164]:
X = np.array(padded_sequences)
X.shape

(1255, 39, 1662)

In [165]:
y = np.array(labels)

In [166]:
y.shape

(1255,)

In [167]:
from keras.utils import to_categorical


In [168]:
y = to_categorical(labels).astype(int)

In [179]:
y.shape

(1255, 114)

In [191]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [192]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape    

((1004, 39, 1662), (251, 39, 1662), (1004, 114), (251, 114))

In [193]:
filtered_words.shape[0]

113

In [231]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LayerNormalization, MultiHeadAttention, Input, Add
from keras.optimizers import Adam

In [235]:
input_shape = (39, 1662)  # 39 timesteps, 1662 features
num_classes = 114

# Transformer

In [15]:
# MatplotLib Global Settings
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['axes.titlesize'] = 24

In [16]:
# If True, processing data from scratch
# If False, loads preprocessed data
PREPROCESS_DATA = True
TRAIN_MODEL = True
# True: use 10% of participants as validation set
# False: use all data for training -> gives better LB result
USE_VAL = False

N_ROWS = 543
N_DIMS = 3
DIM_NAMES = ['x', 'y', 'z']
SEED = 42
NUM_CLASSES = 100
IS_INTERACTIVE = True
VERBOSE = 1 if IS_INTERACTIVE else 2

INPUT_SIZE = 64

BATCH_ALL_SIGNS_N = 4
BATCH_SIZE = 32
N_EPOCHS = 100
LR_MAX = 1e-3
N_WARMUP_EPOCHS = 0
WD_RATIO = 0.05
MASK_VAL = 4237

In [17]:
def print_shape_dtype(l, names):
    for e, n in zip(l, names):
        print(f'{n} shape: {e.shape}, dtype: {e.dtype}')